**Install libraries**

In [1]:
! pip install strip-markdown cohere crewai==0.28.8 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.9/168.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
#Control warnings
import warnings
warnings.filterwarnings('ignore')

Import from the crewAI libray.

In [3]:
from crewai import Agent, Task , Crew

In [4]:
import os
from langchain_community.chat_models import ChatCohere
os.environ["COHERE_API_KEY"] ="XXXXXXXXXX"   # replace with your actual API KEY
llm = ChatCohere()

**Creating Agents**

**Agent: Planner**

In [5]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
               "about the topic: {topic}." \
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
allow_delegation = False ,
     llm =llm,
     verbose= True
)

**Agent: Writer**

In [6]:
writer = Agent (
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation = False,
    llm = llm,
    verbose = True


)

**Agent: Editor**

In [7]:
editor = Agent (
     role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
     allow_delegation = False,
     llm = llm ,
     verbose = True


)

**Creating Tasks:**

**Task: Plan**

In [8]:
plan = Task (
    description=
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources.",
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent = planner

)

**Task: Write**

In [9]:
write = Task(
    description=
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n",
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent = writer

)

**Task: Edit**

In [10]:
edit= Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent = editor

)

**Creating the Crew**

In [11]:
crew = Crew (
    agents = [planner, writer, editor],
    tasks = [plan , write, edit],
    verbose = 2
)

**Running the Crew**

In [12]:
result = crew.kickoff({"topic": "Intelligent Agent"})
print(crew.usage_metrics)

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Intelligent Agent.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer

Final Answer: 

## Content Plan: Intelligent Agents 

### Trend Prioritization:
- Latest Trends:
   - Hybrid Intelligent Agents: Combining the strengths of artificial intelligence and human expertise to create adaptive and context-aware systems.
   - Autonomous Vehicles and Drones: Intelligent agents power self-driving cars and delivery drones, enabling them to navigate and make decisions independently.
   - Virtual Assistants and Chatbots: Developing intelligent agents to provide personalized assistance and conversation

Display the results of your execution as markdown in the notebook.

In [13]:
from markdown import Markdown
import strip_markdown

# Convert to HTML first (if your markdown library doesn't directly output plain text)
markdown = Markdown()
html_text = markdown.convert(result)

# Strip the HTML tags
text = strip_markdown.strip_markdown(html_text)

# Print the plain text
print(text)

Unlocking the Potential of Intelligent Agents: Powering the Future of Decision-Making
Introduction: Embracing the Age of Intelligent Agents
We are witnessing a remarkable era in the evolution of technology with the advent of intelligent agents. From self-driving cars to the ever-present virtual assistants on our devices, intelligent agents are an integral part of our daily lives. But what exactly are these agents, and why are they generating so much excitement?
Intelligent agents are autonomous and adaptable entities, defined by their capacity for independent decision-making and problem-solving. They possess key attributes such as autonomy, freeing them from constant human oversight, and adaptability, allowing them to learn and adjust to new environments. These traits set intelligent agents apart from conventional software, as they emulate human cognitive abilities.
As we delve into the world of intelligent agents, we will explore their far-reaching impact on various sectors, examine t